# **装包和数据**

In [ ]:
from tableone import TableOne, load_dataset
import pandas as pd
import torch
import os
import random
import pyreadstat
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn import metrics
from matplotlib.pyplot import cm
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix, roc_curve
)
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import cupy as cp
import xgboost as xgb
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
print(torch.__version__)
print(torch.zeros(1).cuda())
print(torch.cuda.is_available())

In [ ]:
os.path.abspath(os.getcwd())

特征工程

In [ ]:
# -------------------- 3.1 定义分类特征 -----------------------
# 分类特征分为二分类和多分类
binary_features = [
    'Sex', 
    'history of hypertension', 
    'history of diabetes', 
    'elevated blood glucose',
    'history of hyperlipidemia', 
    'coronary heart disease', 
    'stroke',
    'history of tumor', 
    'hypertensive medications', 
    'lipid-lowering drugs', 
    'glucose-lowering drugs'
]

multi_class_features = [
    'smoking', 
    'alcohol'
]

# 合并所有分类特征
categorical_features = binary_features + multi_class_features

In [ ]:
# -------------------- 3.2 独立记录特征 (Record-Level) -----------------------
X_independent = train_df.drop(columns=['NAFLD', 'PatientID'])  # 这里保留 'Nth_record_for_patient' 和其他特征
y_independent = train_df['NAFLD']

In [ ]:
# -------------------- 3.3 患者历史记录特征 (Patient-Level) -----------------
# 定义加权聚合函数
def exponential_weighted_feature(grp, alpha=0.7):
    """
    为每位患者的每条记录计算指数加权移动平均（EWMA）特征。
    
    参数：
    - grp: 按PatientID分组后的单个子DataFrame。
    - alpha: 权重衰减因子，值越大，近期记录权重越大。
    
    返回：
    - 带有新加权特征的DataFrame。
    """
    # 按ExamYear排序
    grp = grp.sort_values('ExamYear')
    
    # 对每个数值特征应用EWMA
    numeric_features = grp.select_dtypes(include=[np.number]).columns.tolist()
    # 排除 'Nth_record_for_patient' 和 'ExamYear'（如果不需要）
    if 'Nth_record_for_patient' in numeric_features:
        numeric_features.remove('Nth_record_for_patient')
    if 'ExamYear' in numeric_features:
        numeric_features.remove('ExamYear')
    if 'PatientID' in numeric_features:
        numeric_features.remove('PatientID')
    
    for feature in numeric_features:
        # ewma_col = f'{feature}_ewma'
        grp[feature] = grp[feature].ewm(alpha=alpha).mean()
    
    return grp

# 应用加权聚合
df_sorted = train_df.sort_values(by=['PatientID', 'ExamYear'])
df_ewm = df_sorted.groupby('PatientID').apply(exponential_weighted_feature, alpha=0.7).reset_index(drop=True)

In [ ]:
# df_ewm column names
df_ewm.columns

In [ ]:
# ------------------------- 4.1 计算患者历史记录数量并生成动态权重 -------------------------
# 根据数据 test_df 中每位患者出现的次数来决定“独立记录模型”与“患者记录模型”的权重
patient_record_counts = test_df.groupby('PatientID').size()
max_records = patient_record_counts.max()
min_records = patient_record_counts.min()

# 使用对数缩放计算权重，以平滑权重分布
patient_weights = np.log1p(patient_record_counts - min_records) / np.log1p(max_records - min_records)

# 计算 alpha 值（独立记录模型的权重）
alpha = 1 - patient_weights

# 将 alpha 映射回每条独立记录
alpha_full = test_df['PatientID'].map(alpha)

模型训练和预测

In [ ]:
# 计算 train_df 当中阳性样本和阴性样本的比例以用于 scale_pos_weight 参数
# 计算正类和负类的数量
num_positive = np.sum(train_df['NAFLD'] == 1)  # 患病样本的数量
num_negative = np.sum(train_df['NAFLD'] == 0)  # 无病样本的数量

# 计算 scale_pos_weight
scale_pos_weight = num_negative / num_positive

In [ ]:
scale_pos_weight

In [ ]:
# ------------------------- 5.1 独立记录模型 (模型1) -------------------------
# 定义列变换器
# 检查是否有GPU可用
gpu_available = cp.cuda.is_available()

# 定义列变换器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # 保留其他数值特征
)

# 创建管道
pipeline_independent = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        use_label_encoder=False, 
        eval_metric='logloss', 
        random_state=42,
        tree_method='gpu_hist' if gpu_available else 'auto',  # 启用GPU计算
        gpu_id=0 if gpu_available else None,  # 如果有GPU可用，指定GPU ID
        scale_pos_weight=scale_pos_weight,  # 根据数据不平衡情况调整此值
        enable_categorical=True
    ))
])

# 使用 GroupKFold 进行交叉验证（确保每个患者的所有记录在同一折中）
group_cv = GroupKFold(n_splits=5)

# 设置GridSearchCV来寻找最佳超参数
param_grid = {
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__subsample': [0.7, 0.8, 0.9],
    'classifier__colsample_bytree': [0.7, 0.8, 0.9]
}

# 初始化 GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline_independent,
    param_grid=param_grid,
    scoring='roc_auc',  # 使用AUC作为评价指标
    cv=group_cv,  # 使用GroupKFold进行分组交叉验证
    verbose=1,
    n_jobs=-1
)

# 训练GridSearchCV，传递'groups'参数
Ags = grid_search.fit(train_df.drop(columns=['NAFLD', 'PatientID','ExamYear']), train_df['NAFLD'], groups=train_df['PatientID'])

# 输出最优超参数
print(f"Best Parameters for Independent Model: {Ags.best_params_}")

# 获取最佳模型
best_model_independent = Ags.best_estimator_

# 使用最佳模型在全量数据上进行训练
best_independent = best_model_independent.fit(train_df.drop(columns=['NAFLD', 'PatientID','ExamYear']), train_df['NAFLD'])

# 预测概率
y_pred_independent = best_independent.predict_proba(test_df.drop(columns=['NAFLD','PatientID','ExamYear']))[:, 1]

# 保存最佳模型
joblib.dump(best_independent, 'best_independent_model.pkl')

# 加载最佳模型
# best_model_independent = joblib.load('best_independent_model.pkl')

In [ ]:
# -------------------- 3.1 定义分类特征 -----------------------
# 分类特征分为二分类和多分类
binary_features = [
    'Sex', 
    'history of hypertension', 
    'history of diabetes', 
    'elevated blood glucose',
    'history of hyperlipidemia', 
    'coronary heart disease', 
    'stroke',
    'history of tumor', 
    'hypertensive medications', 
    'lipid-lowering drugs', 
    'glucose-lowering drugs'
]

multi_class_features = [
    'smoking', 
    'alcohol'
]

# 合并所有分类特征
categorical_features = binary_features + multi_class_features
categorical_features

In [ ]:
# ------------------------- 5.2 患者记录模型 (模型2) ------------------------- #
# 判断是否有GPU可用
gpu_available = cp.cuda.is_available()
# 定义列变换器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # 保留其他数值特征
)

# 创建管道
pipeline_patient = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        use_label_encoder=False, 
        eval_metric='logloss', 
        random_state=42,
        tree_method='gpu_hist' if gpu_available else 'auto',  # 启用GPU计算
        gpu_id=0 if gpu_available else None,  # 如果有GPU可用，指定GPU ID
        scale_pos_weight=scale_pos_weight,  # 根据数据不平衡情况调整此值
        enable_categorical=True
    ))
])

# 使用 GroupKFold 进行交叉验证（确保每个患者的所有记录在同一折中）
group_cv = GroupKFold(n_splits=5)

# 设置GridSearchCV来寻找最佳超参数
param_grid = {
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__subsample': [0.7, 0.8, 0.9],
    'classifier__colsample_bytree': [0.7, 0.8, 0.9]
}

# 初始化 GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline_patient,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=group_cv,
    verbose=1,
    n_jobs=-1
)

# 训练 GridSearchCV，传递 'groups' 参数
Bgs = grid_search.fit(df_ewm.drop(columns=['NAFLD', 'PatientID','ExamYear']), df_ewm['NAFLD'], groups=df_ewm['PatientID'])

print(f"Best Parameters for Patient Model: {grid_search.best_params_}")
best_model_patient = Bgs.best_estimator_

# 在全量训练数据上训练最佳模型
best_patient = best_model_patient.fit(df_ewm.drop(columns=['NAFLD', 'PatientID','ExamYear']), df_ewm['NAFLD'])

In [ ]:
# 预测患者级的概率
y_pred_patient_full = best_patient.predict_proba(test_df.drop(columns=['NAFLD', 'PatientID','ExamYear']))[:, 1]

# 使用 joblib 保存最佳模型
joblib.dump(best_patient, 'best_patient_model.pkl')

# 加载最佳模型（如果需要）
# best_model_patient = joblib.load('best_patient_model.pkl')

In [ ]:
# ------------------------- 6.1 动态加权融合预测 ------------------------- #
# 确保以下变量已存在：
# y_pred_independent: 独立记录模型的预测概率
# y_pred_patient_full: 患者记录模型的预测概率
# alpha_full: 每条记录的权重

# 计算最终融合概率
y_pred_combined = alpha_full * y_pred_independent + (1 - alpha_full) * y_pred_patient_full

In [ ]:
# ------------------------- 7.1 模型评估和可视化 ------------------------- #
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix, roc_curve
)

# 计算不同阈值下的指标变化
thresholds_plot = np.linspace(0, 1, 100)
metrics = {'Threshold': [], 'Accuracy': [], 'Sensitivity': [], 'Specificity': [], 'AUROC': []}

for thresh in thresholds_plot:
    y_pred_thresh = (y_pred_combined >= thresh).astype(int)
    cm_thresh = confusion_matrix(test_df['NAFLD'], y_pred_thresh)
    if cm_thresh.size == 1:
        tn, fp, fn, tp = 0, 0, 0, cm_thresh[0,0]
    elif cm_thresh.shape == (2, 2):
        tn, fp, fn, tp = cm_thresh.ravel()
    else:
        tn, fp, fn, tp = 0, 0, 0, 0  # 根据实际情况调整
    
    accuracy_thresh = accuracy_score(test_df['NAFLD'], y_pred_thresh)
    sensitivity_thresh = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity_thresh = tn / (tn + fp) if (tn + fp) != 0 else 0
    auroc_thresh = roc_auc_score(test_df['NAFLD'], y_pred_combined)  # 计算AUROC

    # 将结果存储到字典中
    metrics['Threshold'].append(thresh)
    metrics['Accuracy'].append(accuracy_thresh)
    metrics['Sensitivity'].append(sensitivity_thresh)
    metrics['Specificity'].append(specificity_thresh)
    metrics['AUROC'].append(auroc_thresh)

# 转换为DataFrame
metrics_df = pd.DataFrame(metrics)

# 绘制指标变化曲线
plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Threshold'], metrics_df['Accuracy'], label='Accuracy')
plt.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], label='Sensitivity')
plt.plot(metrics_df['Threshold'], metrics_df['Specificity'], label='Specificity')
plt.plot(metrics_df['Threshold'], metrics_df['AUROC'], label='AUROC', linestyle='--')  # 添加AUROC的变化曲线
plt.xlabel('Threshold')
plt.ylabel('Metric Value')
plt.title('Performance Metrics at Different Thresholds')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# get the threshold where the sum of sensitivity and specificity and accuracy and auroc is maximized
metrics_df['Sum'] = metrics_df['Accuracy'] + metrics_df['Sensitivity'] + metrics_df['Specificity'] + metrics_df['AUROC']
max_sum_idx = metrics_df['Sum'].idxmax()
best_threshold = metrics_df['Threshold'][max_sum_idx]
print(best_threshold)

In [ ]:
# ------------------------- 7.2 最优Threshold模型评估和可视化 ------------------------- #
# 使用 0.5 作为阈值进行二分类预测
y_pred_labels = (y_pred_combined >= best_threshold).astype(int)

# 计算混淆矩阵
cm = confusion_matrix(test_df['NAFLD'], y_pred_labels)
if cm.shape == (1, 1):
    tn, fp, fn, tp = 0, 0, 0, cm[0,0]
elif cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
else:
    tn, fp, fn, tp = 0, 0, 0, 0  # 根据实际情况调整

# 计算各项指标
accuracy = accuracy_score(test_df['NAFLD'], y_pred_labels)
auroc = roc_auc_score(test_df['NAFLD'], y_pred_combined)
sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

print("=== Evaluation Results ===")
print(f"Accuracy   : {accuracy:.4f}")
print(f"AUROC      : {auroc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

# 绘制ROC曲线
import matplotlib.pyplot as plt

fpr, tpr, thresholds_roc = roc_curve(test_df['NAFLD'], y_pred_combined)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUROC = {auroc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # 随机猜测的参考线
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC Curve')
plt.grid(True)
plt.show()


8.特征重要性分析

In [ ]:
# ------------------------- 8.1 独立记录模型的特征重要性 -------------------------
# 获取训练后的最佳模型
model_independent_final = best_independent.named_steps['classifier']

all_features = best_independent.named_steps['preprocessor'].get_feature_names_out()
all_features = [name.replace('cat__', '').replace('remainder__', '') for name in all_features]

# 获取特征重要性
importance_independent = model_independent_final.feature_importances_

# 检查特征数是否一致
assert len(all_features) == len(importance_independent), \
    f"Feature names count ({len(all_features)}) does not match importance count ({len(importance_independent)})"

# 创建特征重要性DataFrame
feat_importance_independent = pd.DataFrame({
    'Feature': all_features,
    'Importance': importance_independent
}).sort_values(by='Importance', ascending=True)  # 排序，按重要性降序

# 显示结果
import matplotlib.pyplot as plt

# 绘制特征重要性
plt.figure(figsize=(10, 12))
bars = plt.barh(feat_importance_independent['Feature'], feat_importance_independent['Importance'])
plt.xlabel('Feature Importance')
plt.title('Feature Importance (Independent Record Model)')

# 在条形图末尾添加每个特征的重要性数值
for bar in bars:
    width = bar.get_width()  # 获取条形的宽度，即特征的重要性值
    plt.text(width, bar.get_y() + bar.get_height() / 2, f'{width:.4f}', va='center')  # va='center' 让文本垂直居中

plt.show()

In [ ]:
# ------------------------- 8.2 患者记录模型的特征重要性 -------------------------
# 获取训练后的最佳模型
model_patient_final = best_patient.named_steps['classifier']

all_features = best_patient.named_steps['preprocessor'].get_feature_names_out()
all_features = [name.replace('cat__', '').replace('remainder__', '') for name in all_features]

# 获取特征重要性
importance_Patient = model_patient_final.feature_importances_

# 检查特征数是否一致
assert len(all_features) == len(importance_Patient), \
    f"Feature names count ({len(all_features)}) does not match importance count ({len(importance_Patient)})"

# 创建特征重要性DataFrame
feat_importance_Patient = pd.DataFrame({
    'Feature': all_features,
    'Importance': importance_Patient
}).sort_values(by='Importance', ascending=True)  # 排序，按重要性降序

# 显示结果
import matplotlib.pyplot as plt

# 绘制特征重要性
plt.figure(figsize=(10, 12))
bars = plt.barh(feat_importance_Patient['Feature'], feat_importance_Patient['Importance'])
plt.xlabel('Feature Importance')
plt.title('Feature Importance (Patient Record Model)')

# 在条形图末尾添加每个特征的重要性数值
for bar in bars:
    width = bar.get_width()  # 获取条形的宽度，即特征的重要性值
    plt.text(width, bar.get_y() + bar.get_height() / 2, f'{width:.4f}', va='center')  # va='center' 让文本垂直居中

plt.show()

In [ ]:
# ------------------------- 8.3 特征重要性数据框 -------------------------
# 合并两个模型的特征重要性
feat_importance_combined = pd.merge(
    feat_importance_independent, feat_importance_Patient,
    on='Feature', suffixes=('_Independent', '_Patient')
)

# 计算两个模型的平均重要性
feat_importance_combined['Average_Importance'] = (feat_importance_combined['Importance_Independent'] + feat_importance_combined['Importance_Patient']) / 2

# 按平均重要性排序
feat_importance_combined = feat_importance_combined.sort_values(by='Average_Importance', ascending=False)

# 显示结果
feat_importance_combined

# 保存feat_importance_combined为xlxs
feat_importance_combined.to_excel('feat_importance_combined.xlsx', index=False)

10. 添加基线模型进行结果比较

In [ ]:
# -------------------- 3.1 定义分类特征 -----------------------
# 分类特征分为二分类和多分类
binary_features = [
    'Sex', 
    'history of hypertension', 
    'history of diabetes', 
    'elevated blood glucose',
    'history of hyperlipidemia', 
    'coronary heart disease', 
    'stroke',
    'history of tumor', 
    'hypertensive medications', 
    'lipid-lowering drugs', 
    'glucose-lowering drugs'
]

multi_class_features = [
    'smoking', 
    'alcohol'
]

# 合并所有分类特征
categorical_features = binary_features + multi_class_features
categorical_features

In [ ]:
# ------------------------- 10.1 基线模型 -------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import numpy as np

# 假设 train_df 和 test_df 已经定义，并且都包含 'PatientID' 和目标列 'NAFLD'
# 请根据实际情况调整列名
X_train = train_df.drop(columns=['NAFLD', 'PatientID','ExamYear'])
y_train = train_df['NAFLD']
X_test = test_df.drop(columns=['NAFLD', 'PatientID','ExamYear'])
y_test = test_df['NAFLD']

# 定义列变换器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # 保留其他数值特征
)

# 定义基线模型，包括新增的 XGBoost 模型
baseline_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'Neural Network': MLPClassifier(max_iter=1000, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# 用于存储交叉验证（CV）在训练集上的预测概率和在测试集上的预测概率
baseline_cv_predictions = {}
baseline_test_predictions = {}

# 使用 train_df 中的 PatientID 作为分组进行 GroupKFold 划分
group_kfold = GroupKFold(n_splits=5)

for model_name, model in baseline_models.items():
    print(f"Training {model_name} with cross-validation...")
    # 初始化交叉验证预测概率数组
    cv_pred = np.zeros(len(X_train))
    
    # 交叉验证
    for train_idx, valid_idx in group_kfold.split(X_train, y_train, groups=train_df['PatientID']):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        
        # 定义 Pipeline：预处理 + 模型
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        
        jixian = pipeline.fit(X_tr, y_tr)
        cv_pred[valid_idx] = pipeline.predict_proba(X_val)[:, 1]
    
    baseline_cv_predictions[model_name] = cv_pred
    
    jixian_final = jixian.fit(X_train, y_train)
    test_pred = jixian_final.predict_proba(X_test)[:, 1]
    baseline_test_predictions[model_name] = test_pred

    # 可选：打印测试集的 ROC-AUC 分数
    test_auc = roc_auc_score(y_test, test_pred)
    print(f"{model_name} Test ROC-AUC: {test_auc:.4f}")

In [ ]:
# ------------------------- 10.2 基线模型结果 -------------------------

from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import numpy as np

# 定义一个函数来计算特异度（Specificity），避免与全局变量冲突
def calc_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp)

# 初始化评估结果字典
evaluation_results = {
    'AUROC': {},
    'Accuracy': {},
    'Recall': {},
    'Specificity': {}
}

# 评估每个基线模型的指标
for model_name, y_pred in baseline_test_predictions.items():
    print(f"\n=== Evaluating {model_name} ===")
    
    # 计算 AUROC
    auc = roc_auc_score(test_df['NAFLD'], y_pred)
    
    # 将预测值转化为标签（0 或 1），用于计算 Accuracy、Recall、F1 和 Specificity
    y_pred_label = (y_pred >= best_threshold).astype(int) # 使用与组合模型相同的阈值以确保公平比较
    
    # 计算 Accuracy、Recall、F1 Score 和 Specificity
    acc = accuracy_score(test_df['NAFLD'], y_pred_label)
    rec = recall_score(test_df['NAFLD'], y_pred_label)
    spec = calc_specificity(test_df['NAFLD'], y_pred_label)
    
    # 存储各个评价指标
    evaluation_results['AUROC'][model_name] = auc
    evaluation_results['Accuracy'][model_name] = acc
    evaluation_results['Recall'][model_name] = rec
    evaluation_results['Specificity'][model_name] = spec

# 纳入之前自创模型 "xgboost combined" 的结果
# 假设全局变量 accuracy, auroc, sensitivity, specificity 已经在代码最开始处定义
evaluation_results['AUROC']['XGBoost Combined'] = auroc
evaluation_results['Accuracy']['XGBoost Combined'] = accuracy
evaluation_results['Recall']['XGBoost Combined'] = sensitivity  # sensitivity 与 Recall 等价
evaluation_results['Specificity']['XGBoost Combined'] = specificity

# 打印评估结果
for metric in evaluation_results:
    print(f"\n{metric}:")
    for model_name, score in evaluation_results[metric].items():
        print(f"{metric} - {model_name}: {score:.4f}")

In [ ]:
# ------------------------- 10.3 绘制基线模型的 ROC 曲线 -------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score

plt.figure(figsize=(8, 6))

# 使用 seaborn 的 Set1 色板为基线模型分配颜色
palette = sns.color_palette("Set1", n_colors=len(baseline_test_predictions))

# 绘制各个基线模型的 ROC 曲线，使用不同的颜色
for i, (model_name, y_pred) in enumerate(baseline_test_predictions.items()):
    fpr, tpr, _ = roc_curve(test_df['NAFLD'], y_pred)
    auc_model = roc_auc_score(test_df['NAFLD'], y_pred)
    plt.plot(fpr, tpr, color=palette[i], label=f'{model_name} (AUROC = {auc_model:.4f})')

# 绘制 XGBoost Combined 模型的 ROC 曲线，使用醒目的红色，并加粗
fpr_xgb, tpr_xgb, _ = roc_curve(test_df['NAFLD'], y_pred_combined)
auroc_xgb = roc_auc_score(test_df['NAFLD'], y_pred_combined)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost Combined (AUROC = {auroc_xgb:.4f})',
         linewidth=2.5, color='red')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Baseline Models and XGBoost Combined Model')
plt.legend()
plt.grid(True)
plt.show()


计算模型在不同频次分组下的效果

In [ ]:
# ------------------------- 11.1 整体模型在不同频次分组下的预测效果 -------------------------
# 将test_df分成不同频次的组，按频次<= 2, 3<= 频次 <= 5, 频次 > 5分组
test_patientID_dict_1 = list(map(test_patientID_dict.get, [1,2]))
test_patientID_dict_2 = list(map(test_patientID_dict.get, [3,4,5]))
test_patientID_dict_3 = list(map(test_patientID_dict.get, [6,7,8,9,10,11]))

In [ ]:
test_patientID_dict_1

In [ ]:
print(67767 in result_list_1)
print(67767 in result_list_2)
print(67767 in result_list_3)

In [ ]:
# 将test_patientID_dict_1, test_patientID_dict_2, test_patientID_dict_3里的Int64Index值提取到3个result_list当中
result_list_1 = []
result_list_2 = []
result_list_3 = []
for i in test_patientID_dict_1:
    result_list_1.extend(i.tolist())
for i in test_patientID_dict_2:
    result_list_2.extend(i.tolist())
for i in test_patientID_dict_3:
    result_list_3.extend(i.tolist())

In [ ]:
test_df_1 = test_df[test_df['PatientID'].isin(result_list_1)]
test_df_2 = test_df[test_df['PatientID'].isin(result_list_2)]
test_df_3 = test_df[test_df['PatientID'].isin(result_list_3)]

In [ ]:
# 将每个分组的index存储到3个字典中
test_df_1_index = test_df_1.index
test_df_2_index = test_df_2.index
test_df_3_index = test_df_3.index

# 从y_pred_combined中提取每个分组对应index的预测概率
y_pred_combined_1 = y_pred_combined[test_df_1_index]
y_pred_combined_2 = y_pred_combined[test_df_2_index]
y_pred_combined_3 = y_pred_combined[test_df_3_index]

In [ ]:
# ------------------------- 11.2 整体模型在不同频次分组下的accuracy和auroc -------------------------
# 计算不同频次分组下的accuracy和auroc
accuracy_1 = accuracy_score(test_df_1['NAFLD'], (y_pred_combined_1 >= best_threshold).astype(int))
accuracy_2 = accuracy_score(test_df_2['NAFLD'], (y_pred_combined_2 >= best_threshold).astype(int))
accuracy_3 = accuracy_score(test_df_3['NAFLD'], (y_pred_combined_3 >= best_threshold).astype(int))

auroc_1 = roc_auc_score(test_df_1['NAFLD'], y_pred_combined_1)
auroc_2 = roc_auc_score(test_df_2['NAFLD'], y_pred_combined_2)
auroc_3 = roc_auc_score(test_df_3['NAFLD'], y_pred_combined_3)

# 打印结果
print("=== Results for Different Frequency Groups ===")
print("Accuracy:")
print(f"Group 1: {accuracy_1:.4f}")
print(f"Group 2: {accuracy_2:.4f}")
print(f"Group 3: {accuracy_3:.4f}")
print("\nAUROC:")
print(f"Group 1: {auroc_1:.4f}")
print(f"Group 2: {auroc_2:.4f}")
print(f"Group 3: {auroc_3:.4f}")